<a href="https://colab.research.google.com/github/Stevenn9981/tic_tac_toe/blob/master/tic_tac_toe_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install imageio
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-6).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already th

In [2]:
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb
import random
import os
import shutil
import tempfile
import zipfile

import pygame as pg
from pygame import gfxdraw
from pygame.locals import *

import tensorflow as tf

from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.networks import q_network
from tf_agents.networks import categorical_q_network
from tf_agents.policies import policy_saver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.specs import array_spec
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts

tempdir = "./"

pygame 2.1.3 (SDL 2.0.22, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
tf.version.VERSION

'2.14.0'

## Hyper-parameters

In [4]:
num_iterations = 3000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
initial_collect_episodes = 5  # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
collect_episodes_per_iteration = 1 # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 10  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 30  # @param {type:"integer"}

gamma = 0.99    # @param {type:"number"}
n_step_update = 2  # @param {type:"integer"}
num_atoms = 51  # @param {type:"integer"}
min_q_value = -20  # @param {type:"integer"}
max_q_value = 20  # @param {type:"integer"}
fc_layer_params = (100,)

BOARD_SIZE = 9 # @param {type:"integer"}

REWARD_WIN = 3    # @param {type:"number"}
REWARD_LOSE = -3    # @param {type:"number"}
REWARD_DRAW = 1    # @param {type:"number"}

## Environment

In [5]:
import gym
import numpy as np
import math
from tabulate import tabulate

from typing import Tuple, List


class TicTacToeEnv1(py_environment.PyEnvironment):
    """
    Implementation of a TicTacToe Environment based on OpenAI Gym standards
    This class is modified from https://github.com/MauroLuzzatto/OpenAI-Gym-TicTacToe-Environment
    I modified that to follow the instructions of Question 1.
    """

    def __init__(self, mode='rgb_array') -> None:
        """This class contains a TicTacToe environment for OpenAI Gym

        Args:
            mode (str): render mode: human or rgb_array, which returns a text or RGB array of a picture that shows the current board
        """
        self.n_actions = BOARD_SIZE * BOARD_SIZE         # 9 * 9 grids to drop
        # state: 9 * 9 * 2: the first 9 * 9 layer is the opponent's play history and the second 9 * 9 layer is own.
        self._observation_spec = {'state': array_spec.BoundedArraySpec(shape=(BOARD_SIZE, BOARD_SIZE, 2), dtype=np.int_, minimum=0, maximum=1),
                                  'legal_moves': array_spec.ArraySpec(shape=(self.n_actions,), dtype=np.bool_)}

        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int_, minimum=0, maximum=80)
        self.render_mode = mode
        self.colors = [1, 2]
        self.screen = None
        self.fields_per_side = BOARD_SIZE
        self.reset()

    def observation_spec(self):
        return self._observation_spec

    def action_spec(self):
        return self._action_spec

    def _reset(self) -> Tuple[np.ndarray, dict]:
        """
        reset the board game and state
        """
        self.board: np.ndarray = np.zeros(
            (self.fields_per_side, self.fields_per_side), dtype=int
        )
        self.current_player = 1
        self.info = {"players": {1: {"actions": []}, 2: {"actions": []}}, "Occupied": set(), "legal_moves": np.ones((self.n_actions,), dtype=bool)}

        # return self.decompose_board_to_state()
        observations_and_legal_moves = {'state': self.decompose_board_to_state(), 'legal_moves': self.info["legal_moves"]}
        return ts.restart(observations_and_legal_moves)

    def decompose_board_to_state(self):
        """
        Our state is a 9x9x2 matrix.
        The first layer is the opponent's play history, 0 means no stone, 1 means stones placed by the opponent.
        The second layer is the current player's history, 0 means no stone, 1 means stones placed by the current player.
        """
        opponent = 2 if self.current_player == 1 else 1
        o_plays = (self.board == opponent) * 1
        c_plays = (self.board == self.current_player) * 1
        return np.stack([o_plays, c_plays], axis=2)


    def _step(self, action: int) -> Tuple[np.ndarray, int, bool, dict]:
        """step function of the tictactoeEnv1

        Args:
          action (int): integer between [0, 80], each representing a field on the board

        Returns:
          state (np.array): state of 2 players' history, 0 means no stone, 1 means stones placed by the corresponding player (shape: 9x9x2).
          reward (int): reward of the currrent step
          done (boolean): true, if the game is finished
          (dict): empty dict for future game related information
        """
        action = int(action)
        if not (0 <= action < self.n_actions):
            raise ValueError(f"action '{action}' is not in action_space")

        reward = 0
        (row, col) = self.decode_action(action)

        # If the agent/player does not choose an empty square, randomly select an empty one.
        if self.board[row, col] != 0:
            if len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE:
                print('BOARD IS FULL!')
            raise ValueError('ERROR: Not A LEGAL MOVE (NOT EMPTY)')
            # action = random.choice(list(set(range(BOARD_SIZE * BOARD_SIZE)) - self.info["Occupied"]))
            # (row, col) = self.decode_action(action)
            # reward -= 5 # assign a negative reward if the play-out position is not empty


        # randomly select an adjacent position with probability 1/16
        if random.random() < 0.5:
            adjs = [[-1, -1], [-1, 0], [-1, 1], [0, 1], [0, -1], [1, -1], [1, 0], [1, 1]]
            adj = random.choice(adjs)
            row, col = row + adj[0], col + adj[1]

        if 0 <= row < BOARD_SIZE and 0 <= col < BOARD_SIZE and self.board[row, col] == 0:
            self.board[row, col] = self.current_player  # drop the piece on the field
            win = self._is_win(self.current_player, row, col)

            action = row * BOARD_SIZE + col
            self.info["players"][self.current_player]["actions"].append(action)
            self.info["Occupied"].add(action)
            self.info['legal_moves'][action] = False
        else:
            win = False

        if win:
            reward = REWARD_WIN
        elif len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE: # Draw
            reward = REWARD_DRAW

        done = (win or len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE)
        self.current_player = self.current_player + 1 if self.current_player == 1 else 1
        state = self.decompose_board_to_state()

        observations_and_legal_moves = {'state': state, 'legal_moves': self.info['legal_moves']}
        # return next_s, reward, done, self.info
        if done:
            return ts.termination(observations_and_legal_moves, reward)
        else:
            return ts.transition(observations_and_legal_moves, reward)

    def _is_win(self, color: int, r: int, c: int) -> bool:
        """check if this player results in a winner

        Args:
            color (int): of the player
            r (int): row of the current play
            c (int): column of the current play

        Returns:
            bool: indicating if there is a winner
        """

        # check if four equal stones are aligned (horizontal, verical or diagonal)
        directions = [[0, 1], [1, 0], [1, 1], [1, -1]]

        for direct in directions:
            count = 0
            for offset in range(-3, 4):
                if 0 <= r + offset * direct[0] < 9 and 0 <= c + offset * direct[1] < 9:
                    if self.board[r + offset * direct[0], c + offset * direct[1]] == color:
                        count += 1
                        if count == 4:
                            return True
                    else:
                        count = 0

        return False

    def decode_action(self, action: int) -> List[int]:
        """decode the action integer into a colum and row value

        0 = upper left corner
        8 = lower right corner

        Args:
            action (int): action

        Returns:
            List[int, int]: a list with the [row, col] values
        """
        col = action % BOARD_SIZE
        row = action // BOARD_SIZE
        assert 0 <= col < BOARD_SIZE
        return [row, col]

    def render(self) -> None:
        """render the board

        The following charachters are used to represent the fields,
            '-' no stone
            'O' for player 0
            'X' for player 1

        An example for a 3x3 game board:
            ╒═══╤═══╤═══╕
            │ O │ - │ - │
            ├───┼───┼───┤
            │ - │ X │ - │
            ├───┼───┼───┤
            │ - │ - │ - │
            ╘═══╧═══╧═══╛
        """
        board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=str)
        for ii in range(BOARD_SIZE):
            for jj in range(BOARD_SIZE):
                if self.board[ii, jj] == 0:
                    board[ii, jj] = "-"
                elif self.board[ii, jj] == 1:
                    board[ii, jj] = "X"
                elif self.board[ii, jj] == 2:
                    board[ii, jj] = "O"

        if self.render_mode == "human":
            board = tabulate(board, tablefmt="fancy_grid")
            print(board)
            print("\n")


        width = height = 400

        white = (255, 255, 255)
        line_color = (0, 0, 0)

        os.environ["SDL_VIDEODRIVER"] = "dummy"
        pg.init()

        # Set up the drawing window
        if self.screen is None:
          self.screen = pg.display.set_mode([width + 16, height + 16])

        self.screen.fill(white)
        # drawing vertical lines
        for i in range(10):
          pg.draw.line(self.screen, line_color, (width / BOARD_SIZE * i, 0), (width / BOARD_SIZE * i, height), 2)

        # drawing horizontal lines
        for i in range(10):
          pg.draw.line(self.screen, line_color, (0, height / BOARD_SIZE * i), (width, height / BOARD_SIZE * i), 2)
        pg.display.flip()

        # drawing noughts and crosses
        for i in range(BOARD_SIZE):
          for j in range(BOARD_SIZE):
            if self.board[i, j] == 1: # Draw crosses
              pg.draw.lines(self.screen, line_color, True, [(width / BOARD_SIZE * (j + 0.5) - 10,
                                                        height / BOARD_SIZE * (i + 0.5) - 10),
                                                      (width / BOARD_SIZE * (j + 0.5) + 10,
                                                        height / BOARD_SIZE * (i + 0.5) + 10)], 3)
              pg.draw.lines(self.screen, line_color, True, [(width / BOARD_SIZE * (j + 0.5) - 10,
                                                        height / BOARD_SIZE * (i + 0.5) + 10),
                                                        (width / BOARD_SIZE * (j + 0.5) + 10,
                                                          height / BOARD_SIZE * (i + 0.5) - 10)], 3)
            elif self.board[i, j] == 2: # Draw noughts
              pg.draw.circle(self.screen, line_color, (width / BOARD_SIZE * (j + 0.5), height / BOARD_SIZE * (i + 0.5)), 12, 3)

        board = np.transpose(
                np.array(pg.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )

        return board

In [6]:
def observation_and_action_constraint_splitter(obs):
    return obs['state'], obs['legal_moves']

## Random play test

In [7]:
# py_env = suite_gym.wrap_env(TicTacToeEnv1())
py_env = TicTacToeEnv1()
tf_env = tf_py_environment.TFPyEnvironment(py_env)

In [8]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="480" height="480" controls>
      <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return IPython.display.HTML(tag)

def create_policy_eval_video(policy, filename, fps=2):
    py_env = TicTacToeEnv1()
    tf_env = tf_py_environment.TFPyEnvironment(py_env)
    filename = filename + ".mp4"
    with imageio.get_writer(filename, fps=fps) as video:
      time_step = tf_env.reset()
      video.append_data(py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = tf_env.step(action_step.action)
        video.append_data(py_env.render())

    return embed_mp4(filename)

In [9]:
random_policy = random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(), tf_env.action_spec(),
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter)
create_policy_eval_video(random_policy, "random-agent")

## Agent

In [10]:
train_py_env = TicTacToeEnv1()
eval_py_env = TicTacToeEnv1()

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [11]:
train_env.time_step_spec()

TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': {'legal_moves': TensorSpec(shape=(81,), dtype=tf.bool, name=None),
                 'state': BoundedTensorSpec(shape=(9, 9, 2), dtype=tf.int64, name=None, minimum=array(0), maximum=array(1))},
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

In [12]:
conv_layer_params = [16, 32]
action_tensor_spec = tensor_spec.from_spec(tf_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Conv layers configured with the right
# activation and kernel initializer.
def conv_layer(num_units):
  return tf.keras.layers.Conv2D(
                      filters=num_units,
                      kernel_size=[3, 3],
                      padding="same",
                      data_format="channels_last",
                      activation=tf.nn.leaky_relu,
                      dtype=float)


# QNetwork consists of a sequence of Conv layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
normalization1 = tf.keras.layers.BatchNormalization()
normalization2 = tf.keras.layers.BatchNormalization()
conv_layers = [conv_layer(num_units) for num_units in conv_layer_params]
action_conv = tf.keras.layers.Conv2D(filters=4,
                    kernel_size=[1, 1], padding="same",
                    data_format="channels_last",
                    activation=tf.nn.leaky_relu)
flatten = tf.keras.layers.Flatten()
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential([normalization1] + conv_layers + [action_conv, normalization2, flatten, q_values_layer])


In [13]:
class ConvNestedLayer(tf.keras.layers.Layer):
    def __init__(self, *args, **kwds):
        super().__init__(*args, **kwds)
        self.conv1 = tf.keras.layers.Conv2D(
                      filters=32,
                      kernel_size=[3, 3],
                      padding="same",
                      data_format="channels_last",
                      activation=tf.nn.leaky_relu,
                      dtype=float)
        self.conv2 = tf.keras.layers.Conv2D(
                      filters=64,
                      kernel_size=[3, 3],
                      padding="same",
                      data_format="channels_last",
                      activation=tf.nn.leaky_relu,
                      dtype=float)
        self.norm1 = tf.keras.layers.BatchNormalization()
        self.norm2 = tf.keras.layers.BatchNormalization()
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x):
        x = self.norm1(x)
        x = self.conv1(x)
        x = self.norm2(x)
        x = self.conv2(x)
        x = self.flatten(x)
        return x

In [14]:
categorical_q_net = categorical_q_network.CategoricalQNetwork(
    train_env.observation_spec()['state'],
    train_env.action_spec(),
    num_atoms=num_atoms,
    preprocessing_layers = ConvNestedLayer(),
    fc_layer_params=fc_layer_params)

In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# agent = dqn_agent.DqnAgent(
#     train_env.time_step_spec(),
#     train_env.action_spec(),
#     q_network=q_net,
#     optimizer=optimizer,
#     td_errors_loss_fn=common.element_wise_squared_loss,
#     train_step_counter=train_step_counter)


agent1 = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=categorical_q_net,
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter,
    optimizer=optimizer,
    min_q_value=min_q_value,
    max_q_value=max_q_value,
    n_step_update=n_step_update,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=tf.Variable(0))

agent2 = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=categorical_q_net,
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter,
    optimizer=optimizer,
    min_q_value=min_q_value,
    max_q_value=max_q_value,
    n_step_update=n_step_update,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=tf.Variable(0))

agent1.initialize()
agent2.initialize()

## Policy

In [16]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec(),
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter)

In [17]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):
    time_step = environment.reset()
    episode_return = 0.0
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


def compute_avg_return_battle(environment, policy1, policy2, num_episodes=10):

  total_return_1 = 0.0
  total_return_2 = 0.0
  for _ in range(num_episodes):
    time_step = environment.reset()
    episode_return_1 = 0.0
    episode_return_2 = 0.0
    while not time_step.is_last():
      action_step = policy1.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return_1 += time_step.reward
      if not time_step.is_last():
        action_step = policy2.action(time_step)
        time_step = environment.step(action_step.action)
        episode_return_2 += time_step.reward
    total_return_1 += episode_return_1
    total_return_2 += episode_return_2

  avg_return_1 = total_return_1 / num_episodes
  avg_return_2 = total_return_2 / num_episodes
  return [avg_return_1.numpy()[0], avg_return_2.numpy()[0]]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

In [18]:
print(compute_avg_return(eval_env, agent1.policy, 1))
print(compute_avg_return(eval_env, agent2.policy, 1))
print(compute_avg_return_battle(eval_env, agent1.policy, agent2.policy, 1))

3.0
3.0
[0.0, 3.0]


## Replay Buffer

In [19]:
replay_buffer1 = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent1.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

replay_buffer2 = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent2.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

# def collect_step(environment, policy, replay_buffer):
#     time_step = environment.current_time_step()
#     if time_step.is_last():
#         time_step = environment.reset()
#     action_step = policy.action(time_step)
#     next_time_step = environment.step(action_step.action)
#     traj = trajectory.from_transition(time_step, action_step, next_time_step)

#     # Add trajectory to the replay buffer
#     replay_buffer.add_batch(traj)

def collect_episode(environment, policy1, replay_buffer1, policy2, replay_buffer2):
    time_step1 = train_env.reset()
    trajs_1, trajs_2 = [], []
    done_flag = False

    while not time_step1.is_last():
      action_step1 = agent1.collect_policy.action(time_step1)
      next_time_step1 = train_env.step(action_step1.action)
      traj1 = trajectory.from_transition(time_step1, action_step1, next_time_step1)
      time_step2 = next_time_step1
      trajs_1.append(traj1)

      if not time_step2.is_last():
        action_step2 = agent2.collect_policy.action(time_step2)
        next_time_step2 = train_env.step(action_step2.action)
        traj2 = trajectory.from_transition(time_step2, action_step2, next_time_step2)
        trajs_2.append(traj2)
        time_step1 = next_time_step2
      else:
        if tf.equal(next_time_step1.reward, REWARD_DRAW):
            trajs_2[-1] = trajs_2[-1].replace(reward=next_time_step1.reward)
        elif tf.equal(next_time_step1.reward, REWARD_WIN):
            trajs_2[-1] = trajs_2[-1].replace(reward=-next_time_step1.reward)
        done_flag = True

      if done_flag:
        break

    if not done_flag:
        if tf.equal(time_step1.reward, REWARD_DRAW):
            trajs_1[-1] = trajs_1[-1].replace(reward=time_step1.reward)
        elif tf.equal(time_step1.reward, REWARD_WIN):
            trajs_1[-1] = trajs_1[-1].replace(reward=-time_step1.reward)

    for tsj in trajs_1:
        # print(f'Policy 1: {tsj.reward}')
        replay_buffer1.add_batch(tsj)
    for tsj in trajs_2:
        # print(f'Policy 2: {tsj.reward}')
        replay_buffer2.add_batch(tsj)

    return done_flag # if True, means Player 1 wins, then change the order of players

# def collect_step_for_both(environment, policy1, replay_buffer1, policy2, replay_buffer2):
#     time_step = environment.current_time_step()
#     if time_step.is_last():
#         time_step = environment.reset()
#     action_step = policy1.action(time_step)
#     next_time_step = environment.step(action_step.action)
#     traj = trajectory.from_transition(time_step, action_step, next_time_step)
#     replay_buffer1.add_batch(traj)

#     time_step = next_time_step
#     if time_step.is_last():
#         time_step = environment.reset()
#     action_step = policy2.action(time_step)
#     next_time_step = environment.step(action_step.action)
#     traj = trajectory.from_transition(time_step, action_step, next_time_step)
#     replay_buffer2.add_batch(traj)

for _ in range(initial_collect_episodes):
    change_player_flag = collect_episode(train_env, agent1.collect_policy, replay_buffer1, agent2.collect_policy, replay_buffer2)
    # collect_step(train_env, random_policy, replay_buffer1)
    # collect_step(train_env, random_policy, replay_buffer2)

dataset1 = replay_buffer1.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update + 1).prefetch(3)
iterator1 = iter(dataset1)

dataset2 = replay_buffer2.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update).prefetch(3)
iterator2 = iter(dataset2)

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


## Set up the checkpointer and Policy saver

In [20]:
def create_zip_file(dirname, base_filename):
  return shutil.make_archive(base_filename, 'zip', dirname)

In [21]:
policy_dir_1 = os.path.join(tempdir, 'policy_1')
tf_policy_saver_1 = policy_saver.PolicySaver(agent1.policy)

policy_dir_2 = os.path.join(tempdir, 'policy_2')
tf_policy_saver_2 = policy_saver.PolicySaver(agent2.policy)

## Train the agent

In [ ]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent1.train = common.function(agent1.train)
agent2.train = common.function(agent2.train)

# Reset the train step.
agent1.train_step_counter.assign(0)
agent2.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return_battle(eval_env, agent1.policy, agent2.policy, 1)
returns = [avg_return]

# Reset the environment.
time_step = train_env.reset()

for _ in range(num_iterations):

    # Collect a few episodes using collect_policy and save to the replay buffer.
    for _ in range(collect_episodes_per_iteration):
      if change_player_flag:
          agent1, agent2 = agent2, agent1
      change_player_flag = collect_episode(train_env, agent1.collect_policy, replay_buffer1, agent2.collect_policy, replay_buffer2)
      # collect_step(train_env, agent1.collect_policy, replay_buffer1)
      # collect_step(train_env, agent2.collect_policy, replay_buffer2)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator1)
    train_loss_1 = agent1.train(experience).loss
    experience, unused_info = next(iterator2)
    train_loss_2 = agent2.train(experience).loss

    step = agent1.train_step_counter.numpy()

    if step % log_interval == 0:
      print('step = {0}: loss = {1}'.format(step, (train_loss_1 + train_loss_2) / 2))

    if step % eval_interval == 0:
      avg_return = compute_avg_return_battle(eval_env, agent1.policy, agent2.policy, 5)
      print('step = {0}: Average Return = {1}'.format(step, str(avg_return)))
      tf_policy_saver_1.save(policy_dir_1)
      policy_zip_filename = create_zip_file(policy_dir_1, os.path.join(tempdir, 'exported_policy_1'))
      tf_policy_saver_2.save(policy_dir_2)
      policy_zip_filename = create_zip_file(policy_dir_2, os.path.join(tempdir, 'exported_policy_2'))
      returns.append(avg_return)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


step = 10: loss = 3.853548288345337
step = 20: loss = 3.914017677307129
step = 30: loss = 3.9030117988586426


step = 30: Average Return = [2.4, 0.6]


/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


step = 40: loss = 3.851694107055664
step = 50: loss = 3.907681465148926
step = 60: loss = 3.893800735473633
step = 60: Average Return = [1.8, 1.2]


...

step = 70: loss = 3.893742799758911
step = 80: loss = 3.905400514602661
step = 90: loss = 3.8897597789764404
step = 90: Average Return = [1.8, 1.2]


/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


step = 100: loss = 3.9088611602783203
step = 110: loss = 3.898538827896118
step = 120: loss = 3.8709864616394043
step = 120: Average Return = [1.8, 1.2]


/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


step = 130: loss = 3.841935873031616


## Load the trained model

In [ ]:
saved_policy_1 = tf.saved_model.load(policy_dir_1)
create_policy_eval_video(saved_policy_1, "trained-agent-1")
saved_policy_2 = tf.saved_model.load(policy_dir_2)
create_policy_eval_video(saved_policy_2, "trained-agent-2")

## TODO



*   Check whether all functions are correct
*   Check whether the environment is correctly implement
*   Add the variant in `step` function
*   Test whether the agent always chooses an *empty* square and whether they can go to an adjacent place correctly

